<h3><p align="center">导包</p></h3>

In [1]:
import os
import cv2
import struct
import numpy
import torch, torchvision, os
import torch.nn as nn
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt

ModuleNotFoundError: No module named 'cv2'

<h3><p align="center">加载数据集</p></h3>

In [ ]:
def load_mnist(path='.\mnist', name='train'): 
    '''  
    path:数据集的路径
    name:值为train,代表读取训练集和验证集
    '''
    labels_path = os.path.join(path,'%s-labels.idx1-ubyte'% name)
    images_path = os.path.join(path,'%s-images.idx3-ubyte'% name)
    with open(labels_path, 'rb') as lb:
        _, _ = struct.unpack('>II',lb.read(8))
        labels = numpy.fromfile(lb,dtype=numpy.uint8)
    with open(images_path, 'rb') as img:
        _, num, rows, cols = struct.unpack('>IIII',img.read(16))
        images = numpy.fromfile(img,dtype=numpy.uint8).reshape(num, rows, cols)
    return images, labels
def setup_seed(seed):
     torch.manual_seed(seed)
     torch.cuda.manual_seed_all(seed)
     torch.backends.cudnn.deterministic = True

<h3><p align="center">数据预处理</p></h3>

In [ ]:
class mnist_dataset(Dataset):
    def __init__(self, mode='train', data_path='./'):
        super().__init__()
        assert mode in ['train', 'val', 'test']
        if mode=='train':
            self.images, self.labels = load_mnist(path=data_path, name='train')
            self.images = self.images[:55000]
            self.labels = self.labels[:55000]
        elif mode=='val':
            self.images, self.labels = load_mnist(path=data_path, name='train')
            self.images = self.images[55000:]
            self.labels = self.labels[55000:]
        elif mode=='test':
            self.images, self.labels = load_mnist(path=data_path, name='t10k')
    def __getitem__(self, index):
        image = self.images[index]
        label = self.labels[index]
        image = torch.from_numpy(image).float().unsqueeze(0)/255.0
        label_t = torch.zeros((10))
        label_t[int(label)] = 1.0 # 标签使用onehot编码
        label = torch.tensor(label)
        return {'image':image, 'label':label, 'onehot_label':label_t}
    def __len__(self):
        return len(self.images)

<h3><p align="center">网络模型</p></h3>

In [ ]:
class LeNet(nn.Module):
    def __init__(self, n_classes):
        super().__init__()
        self.feature_extractor = nn.Sequential(            
            nn.Conv2d(in_channels=1, out_channels=20, kernel_size=5, stride=1),
            nn.ReLU(),
            nn.AvgPool2d(kernel_size=2,stride=2),
            nn.Conv2d(in_channels=20, out_channels=50, kernel_size=5, stride=1),
            nn.ReLU(),
            nn.AvgPool2d(kernel_size=2,stride=2),
            nn.Conv2d(in_channels=50, out_channels=500, kernel_size=4, stride=1)
        )
        self.classifier = nn.Linear(in_features=500, out_features=n_classes)
    def forward(self, x):
        x = self.feature_extractor(x)
        x = torch.flatten(x, 1)
        logits = self.classifier(x)
        return logits

<h3><p align="center">训练</p></h3>

In [ ]:
def trainer():
    # 设置随机数种子
    setup_seed(20)
    # 训练时的 batch size
    b_size = 10
    # 训练的总遍历数
    epoches = 20
    # 定义损失函数
    loss_fn = torch.nn.CrossEntropyLoss() 
    # 设置学习率
    learning_rate = 1e-4 
    # 每隔多少batch打印一次
    show_time = 500
    # 装载模型
    model = LeNet(n_classes=10)
    # 优化器
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    # 装载训练集
    train_set = mnist_dataset(mode='train', data_path='./mnist')
    train_dataloader = DataLoader(train_set, batch_size=b_size)
    # 装载验证集，默认batch size为1
    val_set = mnist_dataset(mode='val', data_path='./mnist')
    val_dataloader = DataLoader(val_set, batch_size=1)
    for epoch in range(epoches):
        # 使用训练集训练
        model.train()
        for index, data in enumerate(train_dataloader):
            image = data['image']
            label = data['onehot_label']
            # 清空梯度
            optimizer.zero_grad() 
            # 预测并计算损失
            pred = model(image)
            loss = loss_fn(pred, label)
            # 梯度回传并更新
            loss.backward()
            optimizer.step()
            if index%show_time==0:
                print(f'[epoch]  {epoch}, [batch]  {index}, [loss]  {loss.item()}')
        # 保存模型
        if not os.path.exists('./checkpoint'):
            os.mkdir('./checkpoint')
        torch.save(model.state_dict(), f'./checkpoint/lenet_model_epoch{epoch}.pth')
        # 使用验证集测试
        model.eval()
        count = 0
        for index, data in enumerate(val_dataloader):
            image = data['image']
            label = data['label']
            # 预测
            with torch.no_grad():
                pred = torch.argmax(model(image), dim=1)
            if pred[0]==label[0]:
                count+=1
        print(f'[epoch] {epoch}, val_acc {count/index}')
    return model

<h3><p align="center">测试</p></h3>

In [ ]:
# 装载模型
model = trainer()
# 装载验证集，默认batch size为1
test_set = mnist_dataset(mode='test', data_path='./mnist')
test_dataloader = DataLoader(test_set, batch_size=1)
model.eval()
count = 0
for index, data in enumerate(test_dataloader):
    image = data['image']
    label = data['label']
    with torch.no_grad():
        pred = torch.argmax(model(image), dim=1)
    if pred[0]==label[0]:
        count+=1
# 指定checkpoint
use_epoch = 5
print(f'[epoch] {use_epoch}, test_acc {count/index}')

[epoch]  0, [batch]  0, [loss]  2.306887149810791
[epoch]  0, [batch]  500, [loss]  0.18896833062171936
[epoch]  0, [batch]  1000, [loss]  0.43068045377731323
[epoch]  0, [batch]  1500, [loss]  0.2544028162956238
[epoch]  0, [batch]  2000, [loss]  0.12092254310846329
[epoch]  0, [batch]  2500, [loss]  0.04947734251618385
[epoch] 0, val_acc 0.9635927185437088
[epoch]  1, [batch]  0, [loss]  0.05424345284700394
[epoch]  1, [batch]  500, [loss]  0.04955226927995682
[epoch]  1, [batch]  1000, [loss]  0.2960999310016632
[epoch]  1, [batch]  1500, [loss]  0.07838545739650726
[epoch]  1, [batch]  2000, [loss]  0.03898770362138748
[epoch]  1, [batch]  2500, [loss]  0.006833062972873449
[epoch] 1, val_acc 0.9733946789357871
[epoch]  2, [batch]  0, [loss]  0.026589656248688698
[epoch]  2, [batch]  500, [loss]  0.02717437781393528
[epoch]  2, [batch]  1000, [loss]  0.1867002248764038
[epoch]  2, [batch]  1500, [loss]  0.04716789722442627
[epoch]  2, [batch]  2000, [loss]  0.018790798261761665
[ep

<h3><p align="center">绘制特征图</p></h3>

In [ ]:
# 装载模型
model = LeNet(n_classes=10)
# 指定checkpoint
use_epoch = 5
# 加载模型
model.load_state_dict(torch.load(f'./checkpoint/lenet_model_epoch{use_epoch}.pth'))
# 装载验证集，默认batch size为1
test_set = mnist_dataset(mode='test', data_path='./mnist')
index_image = 10
image = test_set.__getitem__(index_image)['image'].unsqueeze(0)
feature_extractor = model.feature_extractor
print('Network Structure: ')
print(feature_extractor)

Network Structure: 
Sequential(
  (0): Conv2d(1, 20, kernel_size=(5, 5), stride=(1, 1))
  (1): ReLU()
  (2): AvgPool2d(kernel_size=2, stride=2, padding=0)
  (3): Conv2d(20, 50, kernel_size=(5, 5), stride=(1, 1))
  (4): ReLU()
  (5): AvgPool2d(kernel_size=2, stride=2, padding=0)
  (6): Conv2d(50, 500, kernel_size=(4, 4), stride=(1, 1))
)


In [ ]:
if not os.path.exists('./feature_map'):
    os.mkdir('./feature_map')
for i in range(len(feature_extractor)):   
    # 提取第i层的输出（包含激活函数和池化层）
    feature_i = feature_extractor[:i+1](image) 
    # 转换成grid形式
    feature_grid = torchvision.utils.make_grid(feature_i[0].unsqueeze(1),10,normalize=True,padding=0) 
    # 保存图片
    torchvision.utils.save_image(feature_grid,f'./feature_map/{index_image}_{i}.png')
    plt.show()

<h3><p align="center">手写图像测试</p></h3>

In [ ]:
# 装载模型
model = LeNet(n_classes=10)
# 指定checkpoint
use_epoch = 5
# 加载模型
model.load_state_dict(torch.load(f'./checkpoint/lenet_model_epoch{use_epoch}.pth'))
image = cv2.imread('./mytest/test1.png', cv2.COLOR_BGR2RGB)
img = torch.from_numpy(image).float().unsqueeze(0)/255
img = img.unsqueeze(dim = 0)
model.eval()